In [3]:
from transformers import RobertaTokenizer, RobertaModel
import torch

# Carga el tokenizador y el modelo
#tokenizer = BertTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-uncased")
#model = BertModel.from_pretrained("dccuchile/bert-base-spanish-wwm-uncased")

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaModel.from_pretrained('roberta-base')

# Oración de ejemplo
sentence = "Hola, esto es una prueba con BERT."

# Tokeniza la oración
inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True, max_length=128)

# Obtiene los embeddings
with torch.no_grad():
    outputs = model(**inputs)

# Los embeddings de la última capa oculta serían
last_hidden_states = outputs.last_hidden_state

# Embedding del [CLS] token que representa la oración entera
sentence_embedding = last_hidden_states[:, 0, :]

# Embeddings de cada token/palabra en la oración
word_embeddings = last_hidden_states

print("Embedding de la oración:", sentence_embedding)
print("Embeddings de las palabras:", word_embeddings)


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

c:\Users\loque\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\huggingface_cache\hub\models--roberta-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Embedding de la oración: tensor([[-8.6743e-02,  9.0471e-02,  1.3302e-02, -1.0595e-01,  3.9593e-02,
         -1.7688e-02, -4.2758e-02, -2.3705e-03,  3.3589e-02, -1.0090e-01,
         -4.4411e-02, -1.7645e-02,  7.2555e-02, -4.6160e-02,  7.3803e-02,
          2.9888e-02, -5.5485e-02, -2.1723e-02, -2.7182e-02, -3.6914e-02,
         -1.0021e-01,  1.2966e-02, -9.6303e-03,  9.4810e-02, -2.5938e-02,
          8.1167e-02,  8.1004e-02,  8.1191e-02, -6.7600e-02,  5.5690e-02,
         -5.8468e-02, -7.9548e-02,  5.6678e-02,  1.0320e-02,  6.8747e-02,
          1.3197e-01,  4.0268e-02, -2.0073e-02, -4.9358e-02,  9.8385e-03,
         -7.8973e-03,  1.7162e-01, -1.2055e-02,  1.9111e-02,  6.3142e-02,
         -3.0364e-03,  2.0137e-02, -6.0194e-03, -3.0871e-02,  2.8043e-02,
          4.6304e-02,  6.4634e-02, -3.9413e-02,  3.3905e-02, -1.8681e-01,
          4.6870e-02,  5.5762e-02,  7.0085e-02, -3.2284e-02, -8.9200e-02,
         -3.3546e-02, -1.7492e-01, -9.4362e-02, -8.4185e-02,  7.9824e-02,
         -6.4

In [4]:
last_hidden_states.shape

torch.Size([1, 17, 768])

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
import json
# Supongamos que tus datos están en un archivo JSON llamado 'converted_data.jsonl'
with open('../scripts/converted_data.jsonl', 'r') as file:
    data = [json.loads(line) for line in file]

# Convertir los datos en un DataFrame de pandas
df = pd.DataFrame(data)

In [6]:
#rename columns
df.rename(columns={'sentence': 'sentence', 'token': 'target_word', 'complexity': 'complexity'}, inplace=True)
df.head()

,sentence,target_word,complexity
0,"Behold, there came up out of the river seven c...",river,0.000000
1,I am a fellow bondservant with you and with yo...,brothers,0.000000
2,"The man, the lord of the land, said to us, 'By...",brothers,0.050000
3,Shimei had sixteen sons and six daughters; but...,brothers,0.150000
4,"""He has put my brothers far from me.",brothers,0.263889


In [7]:
import torch

device = torch.device("cuda")

model.to(device)


RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(50265, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0-11): 12 x RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dropou

In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Suponiendo que df es tu DataFrame que incluye las columnas 'sentence', 'target_word' y 'complexity'
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

from tqdm.auto import tqdm

def get_embeddings(text_list, tokenizer, model, device):
    """Obtiene embeddings [CLS] para una lista de textos, con barra de progreso."""
    model.eval()
    embeddings = []
    for text in tqdm(text_list, desc="Processing", leave=True):
        encoded_input = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512).to(device)
        with torch.no_grad():
            output = model(**encoded_input)
        embeddings.append(output.last_hidden_state[:, 0, :].squeeze().cpu())
    return torch.stack(embeddings)


def process_dataframe(df, tokenizer, model, device):
    """Procesa un dataframe para obtener embeddings y complejidades."""
    sentence_embeddings = get_embeddings(df['sentence'].tolist(), tokenizer, model, device)
    word_embeddings = get_embeddings(df['target_word'].tolist(), tokenizer, model, device)
    complexities = torch.tensor(df['complexity'].values, dtype=torch.float).unsqueeze(1)
    return torch.cat((sentence_embeddings, word_embeddings, complexities), dim=1)

# Procesar los conjuntos de entrenamiento y prueba
train_data = process_dataframe(train_df, tokenizer, model, device)
test_data = process_dataframe(test_df, tokenizer, model, device)

Processing:   0%|          | 0/6129 [00:00<?, ?it/s]

Processing:   0%|          | 0/6129 [00:00<?, ?it/s]

Processing:   0%|          | 0/1533 [00:00<?, ?it/s]

Processing:   0%|          | 0/1533 [00:00<?, ?it/s]

In [9]:
train_data.shape, test_data.shape

(torch.Size([6129, 1537]), torch.Size([1533, 1537]))

In [10]:
# Entradas de entrenamiento (todas las columnas excepto la última)
train_features = train_data[:, :-1]
# Objetivos de entrenamiento (última columna)
train_targets = train_data[:, -1]

# Entradas de prueba
test_features = test_data[:, :-1]
# Objetivos de prueba
test_targets = test_data[:, -1]

# Asegúrate de que los tensores estén en la CPU para convertirlos a arrays de NumPy
train_features_np = train_features.numpy()
train_targets_np = train_targets.numpy()
test_features_np = test_features.numpy()
test_targets_np = test_targets.numpy()


In [11]:
import tensorflow as tf

def rmse(y_true, y_pred):
    return tf.sqrt(tf.reduce_mean(tf.square(y_true - y_pred)))

In [34]:
import tensorflow as tf

# Definir el modelo
model = tf.keras.Sequential([
    tf.keras.layers.Dense(256, activation='relu', input_shape=(train_features.shape[1],)),
    tf.keras.layers.Dropout(0.15),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1)
])

model.compile(optimizer='adam',
              loss=rmse,
              metrics=['mean_absolute_error'])

model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_20 (Dense)            (None, 256)               393472    
                                                                 
 dropout_8 (Dropout)         (None, 256)               0         
                                                                 
 dense_21 (Dense)            (None, 128)               32896     
                                                                 
 dense_22 (Dense)            (None, 1)                 129       
                                                                 
Total params: 426497 (1.63 MB)
Trainable params: 426497 (1.63 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [48]:
# Entrenamiento del modelo
history = model.fit(train_features_np, train_targets_np,
                    validation_split=0.2,
                    epochs=5,
                    batch_size=32)

Epoch 1/5
154/154 [==============================] - 1s 4ms/step - loss: 0.0869 - mean_absolute_error: 0.0678 - val_loss: 0.1112 - val_mean_absolute_error: 0.0848
Epoch 2/5
154/154 [==============================] - 1s 3ms/step - loss: 0.0871 - mean_absolute_error: 0.0677 - val_loss: 0.1091 - val_mean_absolute_error: 0.0836
Epoch 3/5
154/154 [==============================] - 0s 3ms/step - loss: 0.0872 - mean_absolute_error: 0.0677 - val_loss: 0.1079 - val_mean_absolute_error: 0.0834
Epoch 4/5
154/154 [==============================] - 0s 3ms/step - loss: 0.0888 - mean_absolute_error: 0.0693 - val_loss: 0.1100 - val_mean_absolute_error: 0.0847
Epoch 5/5
154/154 [==============================] - 1s 3ms/step - loss: 0.0868 - mean_absolute_error: 0.0675 - val_loss: 0.1032 - val_mean_absolute_error: 0.0799


In [49]:
# Evaluación del modelo
test_loss, test_mae = model.evaluate(test_features_np, test_targets_np, verbose=2)
print(f"Test RMSE: {test_loss}, Test MAE: {test_mae}")

48/48 - 0s - loss: 0.1020 - mean_absolute_error: 0.0782 - 78ms/epoch - 2ms/step
Test RMSE: 0.10195381194353104, Test MAE: 0.07821858674287796


In [53]:
model.save("modelos/roberta.keras")
